## 정형 데이터와 비정형 데이터
- 정형 데이터(structured data) : CSV, 데이터베이스, 엑셀에 저장하기 쉬운 것
- 비정형 데이터(unstructured data) : CSV, 데이터베이스, 엑셀에 저장하기 어려운 것

#  랜덤 포레스트 Random Forest
앙상블 학습(ensemble learning, 정형 데이터를 모델링할 때 가장 뛰어난 성과를 내는 알고리즘)의 대표 주자
<br>랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듦. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듦

### 랜덤 포레스트가 숲을 구성하는 방법
- 각 트리를 훈련하기 위한 데이터를 만듦 : 부투스트랩 샘플(bootstrap sample)
<br> 부트스트랩 샘플이란, 입력한 훈련 데이터에서 중복을 허용하여 랜덤하게 샘플을 추출하여 훈련 데이터 셋을 만드는 방식. 부트스트랩 샘플은 훈련 셋의 크기와 같게 만듦
- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음, 이 중에서 최선의 분할을 찾음
<br> 분류 모델인 RandomForestClassifier는 각 노드마다 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택하여 사용
<br> 회귀 모델인 RandomForestRegressor은 각 노드에 전체 특성을 사용

- 사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 위의 방식으로 훈련

- 예측
<br> 분류 모델일 경우 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음
<br> 회귀 모델일 경우 단순히 각 트리의 예측을 평균함

### 장점
- 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 셋에 과대적합되는 것을 방지
- 검증 셋과 테스트 셋에서 안정적인 성능을 얻을 수 있음

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target,
                                                                     test_size = 0.2, random_state = 42)

cross_validate() 함수를 이용해 교차 검증 수행
- n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어 사용, 최대한 병렬로 교차 검증 수행
- return_strain_score 매개변수를 True로 지정하면 검증 점수 뿐만 아니라 훈련 셋에 대한 점수도 같이 반환함

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


### 랜덤 포레스트의 매개변수
<br> 1 랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공
- criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등 <br>

2 그 외로 각 결정 트리의 특성 중요도를 취합한 랜덤 포레스트의 특성 중요도를 계산
- feature_importances_

In [3]:
rf.fit(train_input, train_target)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

위의 결과를 살펴보면 '결정 트리'에서 만든 특성 중요도와 다름을 알 수 있음
<br> 이는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문
<br> 그 결과 하나의 특성에 과도하게 집중하지 않고 더 많은 특성이 훈련에 기여할 기여를 얻음
<br> 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움을 줌

3 마지막으로, 랜덤 포레스트 분류기에는 자체적으로 모델을 평가하는 점수를 얻을 수 있음
<br> 부트스트랩 샘플에 포함되지 않고 남는 샘플인 OOB(out of bag) 샘플을 활용해 부트스트랩 샘플로 훈련한 결정 트리를 평가 할 수 있음. (검증 셋의 역할)
- oob_score = True
<br> 각 결정 트리의 OOB 점수를 평균하여 출력함

In [4]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
rf.oob_score_

0.8934000384837406

위의 결과를 살펴보면 교차 검증에서 얻은 점수와 비슷한 결과임을 알 수 있음
<br> 결과적으로 OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 셋에 더 많은 샘플을 사용할 수 있음

<br>

# 엑스트라 트리(Extra Trees)
1. 랜덤 포레스트와의 공통점
- 100개의 결정 트리를 훈련
- 결정 트리가 제공하는 대부분의 매개변수를 지원
- 전체 특성 중 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용
<br>
2. 랜덤 포레스트와의 차이점
- 각 결정 트리를 만들 때 부트스트랩 샘플을 사용하지 않고, 전체 훈련 셋을 사용
- 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할
- splitter = 'random'인 결정트리
- 하나의 결정 트리에서 특성을 무작위로 분할하면 성능이 낮아지겠지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 셋의 점수를 높이는 효과가 있음

In [5]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


엑스트라 트리도 랜덤 포레스트와 마찬가지로 특성 중요도를 제공함

In [6]:
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

<br>

# 그래디언트 부스팅 Gradient Boosting
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
<br>기본적으로 깊이가 3인 결정 트리 100개를 사용
<br> 깊이가 얖은 결정 트리를 사용하기 때문에 결정 트리의 개수를 늘려도 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있으
<br> 경사 하강법을 사용하여 트리를 앙상블에 추가
<br> 분류 - 로지스틱 손실 함수 사용, 회귀 - 평균 제곱 오차 함수 사용

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


=> 결정 트리 개수를 500개로 늘려도 과대적합을 잘 억제하고 있음

In [9]:
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15853457, 0.68010884, 0.1613566 ])

=> 그래디언트 부스팅도 특성 중요도를 제공함

그래디언트 부스팅에는 subsample 이라는 매개변수가 존재
- 트리 훈련에 사용할 훈련 셋의 비율을 결정
- 기본값은 1.0으로 전체 훈련 셋을 사용
- 1보다 작으면 훈련 셋의 일부만 사용 (확률적 경사 하강법이나 미니배치 경사 하강법과 비슷)<br> <br>


but 그래디언트 부스팅에는 n_jobs 매개변수가 없음
- 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

<br>

# 히스토그램 기반 그래디언트 부스팅 (Histogram-based Gradient Boosting)
그래디언트 부스팅의 속도와 성능을 개선한 것

### 원리
- 입력 특성을 256개의 구간으로 나눔 => 따라서 노드를 분할할 때 최적의 분할을 빠르게 찾을 수 있음
- 히스토그램 기반 그래디언트 부스팅은 256개의 구간 중 하나를 떼어 놓고 누락된 값으로 사용 => 따라서 누락된 값 전처리 필요 없음

### HIstGradientBoostingClassfier 클래스의 매개변수
1. max_iter : 트리의 개수를 지정하는데 n_estimator 대신 부스팅 반복 횟수를 지정 => 성능에 영향

In [10]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


### permutation_importance() 함수
특성 중요도를 계산
<br> 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한 지를 계산
<br> 훈련 셋뿐만 아니라 테스트 셋에도 적용할 수 있고, 사이킷런의 추정기 모델에 모두 사용할 수 있음

#### 함수의 매개변수
1. n_repeats 매개변수 : 랜덤하게 섞을 횟수를 지정. 기본값은 5

#### 함수가 반환하는 객체
1. importances_ : 반복하여 얻은 특성 중요도
2. importances_mean : 특성 중요도의 평균
3. importances_std : 특성 중요도의 표준 편차

In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
result.importances_mean

array([0.08876275, 0.23438522, 0.08027708])

<br>

테스트 셋에서의 특성 중요도와 성능

In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
result.importances_mean

array([0.05969231, 0.20238462, 0.049     ])

In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

<br>

# XGBoost
다양한 그래디언트 부스팅 알고리즘을 지원
<br> tree_method = 'hist' : 히스톡스램 기반 그래디언트 부스팅을 사용할 수 있음

In [15]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


<br>

# LightGBM
히스토그램 기반 그래디언트 부스팅  라이브러리

In [18]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
